In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-product-reviews/Reviews.csv


In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional,SimpleRNN,GRU
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df=pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

In [4]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [5]:
df.dropna(inplace=True)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
texts = df['Text']  # The text data you want to train on
labels = df['Score']

**RNN**

In [8]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Get the word index and vocab size
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Adding 1 to account for padding token

# Pad the sequences to ensure uniform input size
max_sequence_length = 100  # You can change this based on your data
train_padded = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_padded, validation_padded, training_label_seq, validation_label_seq = train_test_split(
    train_padded, labels, test_size=0.2, random_state=42)


In [9]:
# Adjust the labels if necessary
training_label_seq = training_label_seq - 1
validation_label_seq = validation_label_seq - 1


In [10]:
# Set hyperparameters
embedding_dim = 128  # Embedding dimension
vocab_size = len(word_index) + 1  # Vocabulary size

# Build the RNN model
model = Sequential()

# Embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(Dropout(0.2))

# First RNN layer
model.add(SimpleRNN(units=embedding_dim, return_sequences=True))
model.add(Dropout(0.2))

# Second RNN layer
model.add(SimpleRNN(units=embedding_dim))
model.add(Dropout(0.2))

# Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Output layer for classification (adjusting to your number of classes, here we assume 5 classes)
model.add(Dense(5, activation='softmax'))

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [11]:
history = model.fit(train_padded, training_label_seq, epochs=10,validation_data=(validation_padded, validation_label_seq),verbose=2)

Epoch 1/10


I0000 00:00:1728237566.978828      98 service.cc:145] XLA service 0x78a950004840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728237566.978885      98 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1728237573.097323      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14210/14210 - 281s - 20ms/step - accuracy: 0.6384 - loss: 1.1208 - val_accuracy: 0.6399 - val_loss: 1.1170
Epoch 2/10
14210/14210 - 271s - 19ms/step - accuracy: 0.6385 - loss: 1.1107 - val_accuracy: 0.6399 - val_loss: 1.1056
Epoch 3/10
14210/14210 - 270s - 19ms/step - accuracy: 0.6393 - loss: 1.1040 - val_accuracy: 0.6399 - val_loss: 1.1051
Epoch 4/10
14210/14210 - 270s - 19ms/step - accuracy: 0.6398 - loss: 1.0983 - val_accuracy: 0.6441 - val_loss: 1.0879
Epoch 5/10
14210/14210 - 270s - 19ms/step - accuracy: 0.6409 - loss: 1.0931 - val_accuracy: 0.6420 - val_loss: 1.0828
Epoch 6/10
14210/14210 - 270s - 19ms/step - accuracy: 0.6416 - loss: 1.0875 - val_accuracy: 0.6432 - val_loss: 1.0729
Epoch 7/10
14210/14210 - 271s - 19ms/step - accuracy: 0.6406 - loss: 1.0638 - val_accuracy: 0.6445 - val_loss: 1.0625
Epoch 8/10
14210/14210 - 271s - 19ms/step - accuracy: 0.6421 - loss: 1.0608 - val_accuracy: 0.6458 - val_loss: 1.0454
Epoch 9/10
14210/14210 - 270s - 19ms/step - accuracy: 0.6416 - loss

Lstm

In [12]:
# Set hyperparameters
embedding_dim = 128  # Embedding dimension

# Build the LSTM model
model = Sequential()

# Embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(Dropout(0.2))

# First LSTM layer
model.add(LSTM(units=embedding_dim, return_sequences=True))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(units=embedding_dim))
model.add(Dropout(0.2))

# Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Output layer for classification (adjust based on the number of classes, assuming 5 classes here)
model.add(Dense(5, activation='softmax'))

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [13]:
history = model.fit(train_padded, training_label_seq, epochs=10,validation_data=(validation_padded, validation_label_seq),verbose=2)

Epoch 1/10
14210/14210 - 269s - 19ms/step - accuracy: 0.6963 - loss: 0.8559 - val_accuracy: 0.7590 - val_loss: 0.6553
Epoch 2/10
14210/14210 - 264s - 19ms/step - accuracy: 0.7777 - loss: 0.5971 - val_accuracy: 0.7879 - val_loss: 0.5756
Epoch 3/10
14210/14210 - 263s - 19ms/step - accuracy: 0.8165 - loss: 0.5006 - val_accuracy: 0.8035 - val_loss: 0.5487
Epoch 4/10
14210/14210 - 263s - 19ms/step - accuracy: 0.8449 - loss: 0.4277 - val_accuracy: 0.8093 - val_loss: 0.5418
Epoch 5/10
14210/14210 - 263s - 19ms/step - accuracy: 0.8666 - loss: 0.3715 - val_accuracy: 0.8101 - val_loss: 0.5873
Epoch 6/10
14210/14210 - 263s - 19ms/step - accuracy: 0.8825 - loss: 0.3294 - val_accuracy: 0.8090 - val_loss: 0.5908
Epoch 7/10
14210/14210 - 263s - 18ms/step - accuracy: 0.8947 - loss: 0.2977 - val_accuracy: 0.8082 - val_loss: 0.6159
Epoch 8/10
14210/14210 - 263s - 19ms/step - accuracy: 0.9035 - loss: 0.2730 - val_accuracy: 0.8099 - val_loss: 0.6760
Epoch 9/10
14210/14210 - 265s - 19ms/step - accuracy: 0.

GRU

In [ ]:
# Set hyperparameters
embedding_dim = 128  # Embedding dimension

# Build the GRU model
model = Sequential()

# Embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(Dropout(0.2))

# First GRU layer
model.add(GRU(units=embedding_dim, return_sequences=True))
model.add(Dropout(0.2))

# Second GRU layer
model.add(GRU(units=embedding_dim))
model.add(Dropout(0.2))

# Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Output layer for classification (adjust based on the number of classes, assuming 5 classes here)
model.add(Dense(5, activation='softmax'))

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [17]:
history = model.fit(train_padded, training_label_seq, epochs=10,validation_data=(validation_padded, validation_label_seq),verbose=2)

Epoch 1/10
14210/14210 - 265s - 19ms/step - accuracy: 0.7383 - loss: 0.7121 - val_accuracy: 0.7768 - val_loss: 0.6013
Epoch 2/10
14210/14210 - 261s - 18ms/step - accuracy: 0.7982 - loss: 0.5467 - val_accuracy: 0.7982 - val_loss: 0.5498
Epoch 3/10
14210/14210 - 260s - 18ms/step - accuracy: 0.8319 - loss: 0.4592 - val_accuracy: 0.8053 - val_loss: 0.5568
Epoch 4/10
14210/14210 - 261s - 18ms/step - accuracy: 0.8573 - loss: 0.3936 - val_accuracy: 0.8111 - val_loss: 0.5640
Epoch 5/10
14210/14210 - 260s - 18ms/step - accuracy: 0.8751 - loss: 0.3475 - val_accuracy: 0.8096 - val_loss: 0.5888
Epoch 6/10
14210/14210 - 260s - 18ms/step - accuracy: 0.8880 - loss: 0.3138 - val_accuracy: 0.8105 - val_loss: 0.6207
Epoch 7/10
14210/14210 - 260s - 18ms/step - accuracy: 0.8972 - loss: 0.2897 - val_accuracy: 0.8086 - val_loss: 0.6341
Epoch 8/10
14210/14210 - 260s - 18ms/step - accuracy: 0.9042 - loss: 0.2707 - val_accuracy: 0.8075 - val_loss: 0.6599
Epoch 9/10
14210/14210 - 260s - 18ms/step - accuracy: 0.